In [110]:
import numpy as np
import matplotlib.pyplot as plt

n=10
C4 = np.zeros(n)
C6 = np.zeros(n)
rho = np.zeros(n)
k = 1
h = 1



In [111]:
def aux_fill():
    for ii in range (n):
        C4[ii] = 1
        C6[ii] = 1
        rho[ii]= -2


def matrix_A():
    A = np.zeros((n,n))
    for ii in range (n):
        A[ii][ii-1] = (-C4[(ii+1)%n] + 4*C4[ii] + C4[ii-1])/(4*h*h)
        A[ii][ii] = -2*C4[ii]/(h*h) - C6[ii]*k*k
        A[ii][(ii+1)%n] = (C4[(ii+1)%n] + 4*C4[ii] - C4[ii-1])/(4*h*h)
    return A


def matrix_B():
    B = np.zeros((n,n))
    for ii in range (n):
        B[ii][ii] = -rho[ii]
    return B

def matrix_C(BB):
    C = np.zeros((n,n))
    for ii in range (n):
        C[ii][ii] = (BB[ii][ii])**(1/2)
    return C

def matrix_C1(CC):
    C1 = np.zeros((n,n))
    for ii in range (n):
        C1[ii][ii] = 1/CC[ii][ii]
    return C1
        


aux_fill()
A = matrix_A()
B = matrix_B()
C = matrix_C(B)
C1 = matrix_C1(C)
P = np.matmul(C1, A)
P = np.matmul(P, C1)
w2, v = np.linalg.eig (P)
w2
#A
#B
#C
#C1

array([-0.5       , -0.69098301, -1.19098301, -1.80901699, -2.5       ,
       -2.30901699, -0.69098301, -1.19098301, -2.30901699, -1.80901699])